In [10]:
#import pakietów
import numpy as np
import pandas as pd
from IPython.display import HTML
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
import pickle
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
import xgboost

In [11]:
#import danych
col_names = ['ID', 'gender', 'city', 'income', 'birth_date', 'application_date', 'requested_amount', 
             'requested_period', 'financial_obligations', 'employer_name', 'account_bank', 'mobile_verification_flag',
             'var_5', 'var_1', 'offer_amount', 'offer_period', 'interest_rate', 'fee', 'offer_monthly_obligation',
             'filled_form_flag', 'device', 'var_2', 'source', 'var_4', 'disbursed_flag', 'latitude', 'longitude', 'age']

dataset = pd.read_csv('clean_data.csv', delimiter = ',', engine='python', header = None, names = col_names, 
                      index_col = 0, skiprows = 1 )

columns_to_drop = ['city', 'birth_date', 'application_date', 'employer_name', 'account_bank']
columns_to_rename = {'ID':'id', 
                     'gender':'cat01', 
                     'income':'num01', 
                     'requested_amount':'num02', 
                     'requested_period':'num03',
                     'financial_obligations':'num04',
                     'mobile_verification_flag':'cat02',
                     'var_5':'cat03',
                     'var_1':'cat04',
                     'offer_amount':'num05',
                     'offer_period':'num06',
                     'interest_rate':'num07',
                     'fee':'num08',
                     'offer_monthly_obligation':'num09',
                     'filled_form_flag':'cat05',
                     'device':'cat06',
                     'var_2':'cat07',
                     'source':'cat08',
                     'var_4':'cat09',
                     'disbursed_flag':'explained',
                     'latitude':'num10',
                     'longitude':'num11',
                     'age':'num12'
}
dataset = dataset.drop(columns_to_drop, axis=1).rename(columns=columns_to_rename)
HTML(dataset.head().to_html())

,id,cat01,num01,num02,num03,num04,cat02,cat03,cat04,num05,num06,num07,num08,num09,cat05,cat06,cat07,cat08,cat09,explained,num10,num11,num12
0,ID000002C20,0,20000.0,300000.0,5.0,0.0,0,2,7,395010.590188,4.0,20.437781,4548.432433,11059.695014,0,1,4,24,3,0.0,28.610000,77.23000,36.738
1,ID002616Q10,0,11000.0,0.0,0.0,0.0,0,2,7,395010.590188,4.0,16.943776,6327.928340,9752.107351,0,0,3,25,3,0.0,9.912292,78.10986,25.604
2,ID000004E40,1,35000.0,200000.0,2.0,0.0,1,14,16,200000.000000,2.0,13.250000,3809.283897,6762.900000,0,1,4,24,4,0.0,18.975000,72.82583,29.485
3,ID000007H20,1,22500.0,600000.0,4.0,0.0,1,2,7,450000.000000,4.0,19.259909,2449.846284,10396.905402,0,1,6,27,3,0.0,30.740000,76.80000,33.878
4,ID000008I30,1,35000.0,1000000.0,5.0,0.0,1,10,7,920000.000000,5.0,18.586374,3933.206761,10028.065563,0,1,6,27,4,0.0,25.880000,86.60000,26.811


In [5]:
X = dataset.drop(['id','explained'], axis=1)
y = dataset['explained']
train_test_split_ratio = 0.25
X_train, X_test , y_train, y_test = train_test_split(X, y, test_size=train_test_split_ratio, random_state=42)
# stratify dorobić

In [6]:
def get_measures(model, X, y):
    y_predict = model.predict(X)
    accuracy = metrics.accuracy_score(y_predict, y)
    precision = metrics.precision_score(y_predict, y)
    recall = metrics.recall_score(y_predict, y)
    f1 = metrics.f1_score(y_predict, y)
    auc = metrics.roc_auc_score(y, y_predict)
    return [accuracy, precision, recall, f1, auc]

In [12]:
def model_grid_search(classifier, param_grid, X, y):
    grid_search = GridSearchCV(classifier, param_grid, cv=4, scoring='roc_auc', n_jobs=3)
    grid_search.fit(X, y)
    return grid_search  

In [8]:
# 1 - Logistic Regression
log_reg_C = [10**c for c in range(4, 11, 1)]
log_reg_class_weight = [{0:1, 1:1}, {0:1, 1:50},{0:1, 1:100},{0:1, 1:150}, {0:1, 1:200}, {0:1, 1:250}, {0:1, 1:300}, 'balanced'] 
log_reg_param_grid = {'C': log_reg_C, 'class_weight': log_reg_class_weight}
log_reg_grid_search = model_grid_search(LogisticRegression(), log_reg_param_grid, X_train, y_train)

with open('log_reg_grid_search.pickle', 'wb') as file:
    pickle.dump(log_reg_grid_search, file)

C:\Users\Ja\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [9]:
print(str(log_reg_grid_search.best_score_))
HTML(pd.DataFrame(log_reg_grid_search.cv_results_)[['param_C', 'param_class_weight', 'mean_train_score', 'mean_test_score']].to_html())

0.8240594061904507


C:\Users\Ja\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\Ja\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\Ja\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\Ja\Anaconda3\lib\site-packages\sklearn\utils\deprecatio

,param_C,param_class_weight,mean_train_score,mean_test_score
0,10000,"{0: 1, 1: 1}",0.737404,0.734922
1,10000,"{0: 1, 1: 50}",0.827045,0.822909
2,10000,"{0: 1, 1: 100}",0.825707,0.822306
3,10000,"{0: 1, 1: 150}",0.825442,0.823411
4,10000,"{0: 1, 1: 200}",0.824786,0.822946
5,10000,"{0: 1, 1: 250}",0.823525,0.820665
6,10000,"{0: 1, 1: 300}",0.823098,0.820796
7,10000,balanced,0.826748,0.822698
8,100000,"{0: 1, 1: 1}",0.751478,0.743488
9,100000,"{0: 1, 1: 50}",0.827094,0.823100


In [ ]:
#2 - DecisionTree
from sklearn.tree import DecisionTreeClassifier

tree_splitter = ['best', 'random']
tree_max_depth = [3, 5, 7, 9, 11, 15, 20, 30, 50]
tree_class_weight = [{0:1, 1:1}, {0:1, 1:50},{0:1, 1:100},{0:1, 1:150}, {0:1, 1:200}, {0:1, 1:250}, {0:1, 1:300}, 'balanced']
tree_param_grid = {'splitter':tree_splitter, 'max_depth':tree_max_depth, 'class_weight':tree_class_weight}
tree_classifier = DecisionTreeClassifier(random_state=42, tree_splitter='best', tree_max_depth=10, class_weight='balanced')

tree_grid_search = model_grid_search(tree_classifier, tree_param_grid, X_train, y_train)

with open('xgb_grid_search.pickle', 'wb') as file:
    pickle.dump(xgb_grid_search, file)


In [13]:
#3 - RandomForest

rf_max_depth = [3, 5, 7, 9, 11]
rf_class_weight = [{0:1, 1:1}, {0:1, 1:50},{0:1, 1:100},{0:1, 1:150}, {0:1, 1:200}, {0:1, 1:250}, {0:1, 1:300}, 'balanced'] 
rf_n_estimators = [100, 300, 500, 700]
rf_param_grid = {'max_depth':rf_max_depth, 'class_weight':rf_class_weight, 'n_estimators':rf_n_estimators}
rf_classifier = RandomForestClassifier(n_estimators=100, max_depth=None, random_state=42, class_weight='balanced')

rf_grid_search = model_grid_search(classifier, rf_param_grid, X_train, y_train)

with open('rf_grid_search.pickle', 'wb') as file:
    pickle.dump(rf_grid_search, file)

In [ ]:
# 4 - Gradient Boosting

from sklearn.emsemble import GradientBoostingClassifier

gb_learning_rate = [0.001, 0.01, 0.1, 1., 10.]
gb_n_estimators = [20, 50, 100, 200, 300, 500, 700]
gb_max_features = ['sqrt', 'log2', None]
gb_param_grid = {'learning_rate':gb_learning_rate, 'n_estimators':gb_n_estimators, 'max_features':gb_max_features, verbose=1) 
gb_classifier = GradientBoostingClassifier(learning_rate = 0.01, n_estimators = 10, random_state=42, max_features=None)

gb_grid_search = model_grid_search(gb_classifier, gb_param_grid, X_train, y_train)

with open('gb_grid_search.pickle', 'wb') as file:
    pickle.dump(gb_grid_search, file)                 
                 

In [ ]:
# 5 - XGBoost

import xgboost

xgb_learning_rate = [0.001, 0.01, 0.1, 1., 10.]
xgb_n_estimators = [20, 50, 100, 200, 300, 500, 700]
xgb_max_features = ['sqrt', 'log2', None]
xgb_param_grid = {'learning_rate':gb_learning_rate, 'n_estimators':gb_n_estimators, 'max_features':gb_max_features, verbose=1) 
xgb_classifier = GradientBoostingClassifier(learning_rate = 0.01, n_estimators = 10, random_state=42, max_features=None)

gb_grid_search = model_grid_search(gb_classifier, gb_param_grid, X_train, y_train)

with open('gb_grid_search.pickle', 'wb') as file:
    pickle.dump(gb_grid_search, file)                 
                 

In [22]:
print(str(rf_grid_search.best_score_))
HTML(pd.DataFrame(rf_grid_search.cv_results_)[['param_max_depth', 'param_class_weight', 'param_n_estimators', 'mean_train_score', 'mean_test_score']].to_html())

0.847183766145


C:\Users\b0622748\Anaconda3_\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\b0622748\Anaconda3_\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\b0622748\Anaconda3_\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\b0622748\Anaconda3_\lib\site-package

,param_max_depth,param_class_weight,param_n_estimators,mean_train_score,mean_test_score
0,3,"{0: 1, 1: 1}",100,0.835318,0.821194
1,3,"{0: 1, 1: 1}",300,0.836660,0.821956
2,3,"{0: 1, 1: 1}",500,0.836778,0.821978
3,3,"{0: 1, 1: 1}",700,0.836659,0.821811
4,5,"{0: 1, 1: 1}",100,0.875789,0.831208
5,5,"{0: 1, 1: 1}",300,0.879076,0.832710
6,5,"{0: 1, 1: 1}",500,0.880166,0.833126
7,5,"{0: 1, 1: 1}",700,0.880371,0.833227
8,7,"{0: 1, 1: 1}",100,0.936967,0.836301
9,7,"{0: 1, 1: 1}",300,0.941869,0.837966


In [ ]:
models = []
log_reg_C = [10**c for c in range(0, 10, 1)]
for C in log_reg_C:
    params = [C, weight]
    model = LogisticRegression(C=C, class_weight=weight)
    model.fit(X_train, y_train)
    models.append([params, model, get_measures(model, X_train, y_train)[7:], get_measures(model, X_test, y_test)[7:]])

In [25]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [32]:
pca = PCA(random_state=42)
pca.fit(X_train_scaled)
explained_variance = pca.explained_variance_ratio_

In [33]:
ratio = 0
explained_variance_cum = []
for item in explained_variance:
    ratio += item
    explained_variance_cum.append(ratio)
explained_variance_cum

[0.15855743150302695,
 0.30531420838780354,
 0.42048994328907063,
 0.48702724773218453,
 0.54202739456214022,
 0.59309743367501244,
 0.64344605074822725,
 0.69166162762600325,
 0.73703175753359429,
 0.7803259063074629,
 0.82043766084475633,
 0.85178636070534752,
 0.87794956356398057,
 0.90233212488147418,
 0.92330951105222248,
 0.94217146341146463,
 0.95851379628331401,
 0.97259168077790747,
 0.98426320775508835,
 0.99365491044237664,
 1.0000000000000002]

In [34]:
pca = PCA(n_components = 14, random_state=42)
pca.fit(X_train_scaled)
explained_variance = pca.explained_variance_ratio_

In [35]:
ratio = 0
explained_variance_cum = []
for item in explained_variance:
    ratio += item
    explained_variance_cum.append(ratio)
explained_variance_cum

[0.1585574315030259,
 0.30531420838780332,
 0.42048994328906975,
 0.48702724773218392,
 0.54202739456213966,
 0.593097433675012,
 0.64344605074822681,
 0.69166162762600281,
 0.73703175753359396,
 0.78032590630746268,
 0.82043766084475611,
 0.85178636070534741,
 0.87794956356398046,
 0.90233212488147407]

In [37]:
X_train_scaled_pca = pca.transform(X_train_scaled)
X_test_scaled_pca = pca.transform(X_test_scaled)

In [39]:
# 1 - Logistic Regression
log_reg_C = [10**c for c in range(4, 11, 1)]
log_reg_class_weight = [{0:1, 1:1}, {0:1, 1:50},{0:1, 1:100},{0:1, 1:150}, {0:1, 1:200}, {0:1, 1:250}, {0:1, 1:300}, 'balanced'] 
log_reg_param_grid = {'C': log_reg_C, 'class_weight': log_reg_class_weight}
log_reg_grid_search = model_grid_search(LogisticRegression(), log_reg_param_grid, X_train_scaled_pca, y_train)

with open('log_reg_grid_search_pca.pickle', 'wb') as file:
    pickle.dump(log_reg_grid_search, file)

In [40]:
print(str(log_reg_grid_search.best_score_))
HTML(pd.DataFrame(log_reg_grid_search.cv_results_)[['param_C', 'param_class_weight', 'mean_train_score', 'mean_test_score']].to_html())

0.821897842616


C:\Users\b0622748\Anaconda3_\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\b0622748\Anaconda3_\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\b0622748\Anaconda3_\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\b0622748\Anaconda3_\lib\site-package

,param_C,param_class_weight,mean_train_score,mean_test_score
0,10000,"{0: 1, 1: 1}",0.812986,0.810100
1,10000,"{0: 1, 1: 50}",0.821748,0.819312
2,10000,"{0: 1, 1: 100}",0.823149,0.820788
3,10000,"{0: 1, 1: 150}",0.823679,0.821354
4,10000,"{0: 1, 1: 200}",0.823932,0.821649
5,10000,"{0: 1, 1: 250}",0.824077,0.821810
6,10000,"{0: 1, 1: 300}",0.824163,0.821898
7,10000,balanced,0.822467,0.820073
8,100000,"{0: 1, 1: 1}",0.812986,0.810100
9,100000,"{0: 1, 1: 50}",0.821748,0.819312


In [ ]:
import matplotlib.pyplot as plt
scores = []
for item in models:
    scores.append(item[2][1])
scores = np.reshape(scores, (15,9))
heatmap(scores, ylabel='C', yticklabels=log_reg_C, xlabel='weight', xticklabels=[item[1] for item in log_reg_class_weight], cmap="viridis")
plt.show()


In [ ]:
scores = np.array(results.mean_test_score).reshape(6, 6)
# plot the mean cross-validation scores
heatmap(scores, xlabel='gamma', xticklabels=param_grid['gamma'], ylabel='C', yticklabels=param_grid['C'], cmap="viridis")
plt.show()

In [ ]:
def heatmap(values, xlabel, ylabel, xticklabels, yticklabels, cmap=None,
            vmin=None, vmax=None, ax=None, fmt="%0.2f"):
    if ax is None:
        ax = plt.gca()
    # plot the mean cross-validation scores
    img = ax.pcolor(values, cmap=cmap, vmin=vmin, vmax=vmax)
    img.update_scalarmappable()
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_xticks(np.arange(len(xticklabels)) + .5)
    ax.set_yticks(np.arange(len(yticklabels)) + .5)
    ax.set_xticklabels(xticklabels)
    ax.set_yticklabels(yticklabels)
    ax.set_aspect(1)

    for p, color, value in zip(img.get_paths(), img.get_facecolors(),
                               img.get_array()):
        x, y = p.vertices[:-2, :].mean(0)
        if np.mean(color[:3]) > 0.5:
            c = 'k'
        else:
            c = 'w'
        ax.text(x, y, fmt % value, color=c, ha="center", va="center")
    return img

In [ ]:
for item in models:
    print(f"C = {item[0][0]}  weight = {item[0][1][1]}  auc_train = {item[2][1]}  auc_test = {item[3][1]}")

In [ ]:
from sklearn.linear_model import LogisticRegression

scores = []
for C in [3**c for c in range(-15, 25, 1)]:
    model = LogisticRegression(C=C)
    model.fit(X_train, y_train)
    scores.append([C, get_measures(model, X_train, y_train), get_measures(model, X_test, y_test)])

C = [item[0] for item in scores]
train = [item[1] for item in scores]
test = [item[2] for item in scores]
df_train = pd.DataFrame(train, columns = ['TN_train', 'FN_train', 'FP_train', 'TP_train', 'accuracy_train', 'precision_train', 'recall_train', 'f1_train', 'auc_train'])
df_test = pd.DataFrame(test, columns = ['TN_test', 'FN_test', 'FP_test', 'TP_test', 'accuracy_test', 'precision_test', 'recall_test', 'f1_test', 'auc_test'])
df = pd.concat([pd.DataFrame(C, columns = ['C']), df_train, df_test], axis=1)
df.to_csv("Linear_regression_scores.csv", sep = ';', index=False)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
import xgboost

xgb_scores = []
for max_depth in [3, 4, 5, 7, 9, 11]:
    for learning_rate in [0.01, 0.1, 0.3, 0.5, 1, 3]:
        for n_estimators in [200, 500, 700, 1000]:
            xgb = xgboost.XGBClassifier(max_depth=max_depth, learning_rate=learning_rate, n_estimators=n_estimators)
            xgb.fit(X_train, y_train)
            xgb_scores.append([ [max_depth, learning_rate, n_estimators], get_measures(model, X_train, y_train), get_measures(model, X_test, y_test)])

hiperparams = [item[0] for item in xgb_scores]
train = [item[1] for item in xgb_scores]
test = [item[2] for item in xgb_scores]
df_params = pd.DataFrame(hiperparams, columns=['max_depth', 'learning_rate', 'n_estimators'])
df_train = pd.DataFrame(train, columns = ['TN_train', 'FN_train', 'FP_train', 'TP_train', 'accuracy_train', 'precision_train', 'recall_train', 'f1_train', 'auc_train'])
df_test = pd.DataFrame(test, columns = ['TN_test', 'FN_test', 'FP_test', 'TP_test', 'accuracy_test', 'precision_test', 'recall_test', 'f1_test', 'auc_test'])
df = pd.concat([df_params, df_train, df_test], axis=1)
df.to_csv("xgboost_scores.csv", sep = ';', index=False)

In [ ]:
def assess_model(model, X_train, y_train, X_test, y_test):
    scores = {}
    confusion_matrix_train = confusion_matrix(model.predict(X_train),y_train)
    confusion_matrix_test = confusion_matrix(model.predict(X_test),y_test)
    accuracy_score_train = accuracy_score(model.predict(X_train),y_train)
    accuracy_score_test = accuracy_score(model.predict(X_test),y_test)
    f1_score_train = f1_score(model.predict(X_train),y_train)
    f1_score_test = f1_score(model.predict(X_test),y_test)

    scores['train'] = {'TN':confusion_matrix_train[0][0], 'FP':confusion_matrix_train[0][1], 
                       'FN':confusion_matrix_train[1][0], 'TP':confusion_matrix_train[1][1],
                        'accuracy':accuracy_score_train, 'F1_score':f1_score_train}

    scores['test'] = {'TN':confusion_matrix_test[0][0], 'FP':confusion_matrix_test[0][1], 
                       'FN':confusion_matrix_test[1][0], 'TP':confusion_matrix_test[1][1],
                        'accuracy':accuracy_score_test, 'F1_score':f1_score_test}
    return pd.DataFrame(scores)

In [ ]:
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
positive = list(X_train[y_train==1].index)
negative = list(X_train[y_train==0].index)

positive_size = 20000
negative_size = 180000

positive_random = np.random.uniform(0, len(positive), positive_size)
positive_random = [ positive[int(x)] for x in positive_random]
positive_random = [X_train[X_train.index == x] for x in positive_random]
positive_random = pd.concat(positive_random)
positive_random['y'] = 1

negative_random = np.random.uniform(0, len(negative), negative_size)
negative_random = [ negative[int(x)] for x in negative_random]
negative_random = [X_train[X_train.index == x] for x in negative_random]
negative_random = pd.concat(negative_random)
negative_random['y'] = 0

X_train_bootstrap = pd.concat([positive_random, negative_random])
X_train_bootstrap = X_train_bootstrap.sample(frac=1).reset_index(drop=True)

y_train_bootstrap = X_train_bootstrap['y']
X_train_bootstrap = X_train_bootstrap.drop(['y'], axis=1)

In [ ]:
bootstrap_scaler = StandardScaler()
bootstrap_scaler.fit(X_train_bootstrap)
X_train_bootstrap_scaled = bootstrap_scaler.transform(X_train_bootstrap)
X_test_bootstrap_scaled = bootstrap_scaler.transform(X_test)

In [ ]:
# sety danych:

plain = X_train, y_train, X_test, y_test
scaled = X_train_scaled, y_train, X_test_scaled, y_test
bootstrap = X_train_bootstrap, y_train_bootstrap, X_test, y_test
bootrstrap_scaled = X_train_bootstrap_scaled, y_train_bootstrap, X_test_bootstrap_scaled, y_test

data_sets = {'plain':plain, 'scaled':scaled, 'boot':bootstrap, 'boot_scal':bootrstrap_scaled}

In [ ]:
def assess_model_plain(model, X_train, y_train, X_test, y_test):
    scores = {}
    confusion_matrix_train = confusion_matrix(model.predict(X_train),y_train)
    confusion_matrix_test = confusion_matrix(model.predict(X_test),y_test)
    accuracy_score_train = accuracy_score(model.predict(X_train),y_train)
    accuracy_score_test = accuracy_score(model.predict(X_test),y_test)
    f1_score_train = f1_score(model.predict(X_train),y_train)
    f1_score_test = f1_score(model.predict(X_test),y_test)

    scores['train'] = {'TN_train':confusion_matrix_train[0][0], 'FP_train':confusion_matrix_train[0][1], 
                       'FN_train':confusion_matrix_train[1][0], 'TP_train':confusion_matrix_train[1][1],
                        'accuracy_train':accuracy_score_train, 'F1_score_train':f1_score_train}

    scores['test'] = {'TN_test':confusion_matrix_test[0][0], 'FP_test':confusion_matrix_test[0][1], 
                       'FN_test':confusion_matrix_test[1][0], 'TP_test':confusion_matrix_test[1][1],
                        'accuracy_test':accuracy_score_test, 'F1_score_test':f1_score_test}
    scores['train'].update(scores['test'])
    return scores['train']

In [ ]:
# Regresja logistyczna

models = []
counter = 1
for data_set in data_sets:
    for C_param in range(-4, 14, 2):
        model = {}
        model['counter'] = counter
        model['data_set'] = data_set
        model['model'] = LogisticRegression(C=10**C_param)
        ds = data_sets[data_set]
        model['model'].fit(ds[0], ds[1])
        model['C'] = C_param
        assessment = assess_model(model['model'], ds[0], ds[1], ds[2], ds[3])
        model.update(assessment)
        models.append(model)
        counter += 1

In [ ]:
[x['train']['F1_score'] for x in models]


In [ ]:
# Regresja logistyczna na bootstrapie
log_reg_bootstrap = LogisticRegression()
log_reg_bootstrap.fit(X_train_bootstrap, y_train_bootstrap)

In [ ]:
assess_model(log_reg_bootstrap, X_train, y_train, X_test, y_test)

In [ ]:
# Regresja logistyczna
log_reg = LogisticRegression()
log_reg_scaled = LogisticRegression()
log_reg.fit(X_train, y_train)
log_reg_scaled.fit(X_train_scaled, y_train)

In [ ]:
assess_model(log_reg, X_train, y_train, X_test, y_test)

In [ ]:
assess_model(log_reg_scaled, X_train_scaled, y_train, X_test_scaled, y_test)

In [ ]:
X_train.head()

In [ ]:
from sklearn.tree import DecisionTreeClassifier
import xgboost

xgb = xgboost.XGBClassifier()
xgb.fit(X_train, y_train)


In [ ]:
from sklearn.tree import DecisionTreeClassifier
import xgboost

xgb_bootstrap = xgboost.XGBClassifier()
xgb_bootstrap.fit(X_train_bootstrap, y_train_bootstrap)
assess_model(xgb_bootstrap, X_train, y_train, X_test, y_test)

In [ ]:
assess_model(xgb, X_train, y_train, X_test, y_test)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
import xgboost

# xgb_bootrstrap_scaled = xgboost.XGBClassifier()
# xgb_bootrstrap_scaled.fit(bootrstrap_scaled[0], bootrstrap_scaled[1])
assess_model(xgb_bootrstrap_scaled, bootrstrap_scaled[0], bootrstrap_scaled[1], bootrstrap_scaled[2], bootrstrap_scaled[3])